In [5]:
import pandas as pd
import h5py
import tqdm
from pathlib import Path

In [6]:
# PRINT COMMANDS TO DOWNLOAD AND EXTRACT DATA

# Set SCRATCH to where you want to download data to
SCRATCH = "/Users/adamhe/github/scratch"

URL = "https://zenodo.org/records/10597358/files"
TAR = "tfmodisco_results.tar.gz"
print(f"wget {URL}/{TAR} -P {SCRATCH}")
print(f"tar -xvzf {SCRATCH}/{TAR} -C {SCRATCH}")

wget https://zenodo.org/records/10597358/files/tfmodisco_results.tar.gz -P /Users/adamhe/github/scratch
tar -xvzf /Users/adamhe/github/scratch/tfmodisco_results.tar.gz -C /Users/adamhe/github/scratch


In [10]:
output = "quantity"
modisco = h5py.File(
    Path(SCRATCH, f"tfmodisco_results/mean_across_folds_all_{output}_modisco.h5")
)
windows = pd.read_csv(
    Path(SCRATCH, "tfmodisco_results/merged_windows_all.bed.gz"), sep="\t", header=None
)
windows.columns = ["chrom", "start", "end"]

In [11]:
example_idx = []
starts = []
ends = []
is_revcomp = []
for i in tqdm.trange(len(modisco["pos_patterns"].keys())):
    example_idx.append(modisco["pos_patterns"][f"pattern_{i}"]["seqlets"]["example_idx"][:])
    starts.append(modisco["pos_patterns"][f"pattern_{i}"]["seqlets"]["start"][:])
    ends.append(modisco["pos_patterns"][f"pattern_{i}"]["seqlets"]["end"][:])
    is_revcomp.append(modisco["pos_patterns"][f"pattern_{i}"]["seqlets"]["is_revcomp"][:])

pos_pattern_coords = []
for i, (idx, start, end, revcomp) in tqdm.tqdm(enumerate(zip(example_idx, starts, ends, is_revcomp))):
    windows_idx = windows.iloc[idx]
    pos_pattern_coords.append(
        pd.DataFrame(
            {
                "chrom": windows_idx["chrom"],
                "start": windows_idx["start"] + start,
                "end": windows_idx["start"] + end,
                "revcomp": revcomp
            }
        ).drop_duplicates()
    )

100%|██████████| 67/67 [00:00<00:00, 578.07it/s]
67it [00:00, 1304.77it/s]


In [12]:
outdir = Path(SCRATCH, "uniq_coords")
outdir.mkdir(exist_ok=True)
for i, coords in tqdm.tqdm(enumerate(pos_pattern_coords)):
    coords.to_csv(
        outdir.joinpath(f"modisco_{output}_pattern_{i}_coords.txt.gz"), sep="\t"
        )

67it [00:00, 197.72it/s]
